In [22]:
import os 
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.schema.runnable import RunnableMap
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.agents import AgentType, initialize_agent
from langchain.agents.agent_toolkits import GmailToolkit
from langchain_core.output_parsers import StrOutputParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.document_loaders.excel import UnstructuredExcelLoader

## 1. Create a model

In [23]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
gmail_toolkit = GmailToolkit()
llm=ChatOpenAI(temperature=0.0,model="gpt-3.5-turbo-1106")

### 2. Load the documents and split it

In [24]:
# Load the document
file_path = "info.xlsx"
loader = UnstructuredExcelLoader(file_path)
docs = loader.load()
# Split the data
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0)
splits = text_splitter.split_documents(docs)



## 3. Create embeddings and retriever

In [26]:

# embedding = OpenAIEmbeddings(model = 'text-embedding-3-large')
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
# To database

db = Chroma.from_documents(documents= splits, embedding = embeddings)
retreiver = db.as_retriever()

prompt = """
You are a virtual assistant who answers users query. When a user asks a question  you have to answer him based on the context given in three back ticks and make sure to provide all the details.
context: ```{context}```
question is given in double back ticks : ``{question}``.
If the questionis out of context, do not provide the wrong answer just say i am sorry i don't have an ansawer to that question.
"""
template = ChatPromptTemplate.from_template(prompt)
output_parser = StrOutputParser()
chain = RunnableMap(
    {
        "context": lambda x:db.similarity_search(x['question'], k =1),
        "question": lambda x:x['question']
    }
)|template|llm|output_parser


### 2. Retrieving mails from ajeetacharya02@gmail.comm

In [40]:

def check_for_mail():
    # prompt = """Your task is to check for new emails and return the output containing only the sender and subject of each email in JSON format"""
    prompt = """You have access to an external system that can check email. Please use it to see if I have received a new email. If there are any, format the response as a JSON object containing only the sender's email address , the subject line of each email and the contents in the email, separated by commas without double inverted commas in the output."""

    agent = initialize_agent(agent= AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
                            tools= gmail_toolkit.get_tools(), 
                            llm= llm, 
                            verbose = False, 
                            max_execution_time = 1600,
                            max_iterations = 1000)
    
    # prompt = PromptTemplate(template=template)
    response = agent.run(prompt.format())
    return response

response = check_for_mail()

email_data = response['emails']
ajeet_data = email_data[1]
ajeet_data_sender = ajeet_data['sender']
ajeet_data_subject = ajeet_data['subject']
ajeet_data_contents = ajeet_data['contents']
ajeet_data

In [47]:
result = chain.invoke({"question":ajeet_data_contents})
result

'Yes, we offer home delivery for all our products. The delivery charge is $3. Shipping typically takes 3-5 business days. If you have any further questions, please feel free to contact our customer service team at +1 (800) 123-4567.'

### 3. Check if there are any matching details about user mail in out database
